In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} 3D visualization of Riemann sheets
TR-999
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2022-rev-resonances.pdf#page=4) of the PDG.
:::
::::

# Riemann sheets

<!-- cspell:disable -->

In [ ]:
%pip install -q ampform==0.14.6 plotly==5.17.0 sympy==1.12

:::{note}
In the following example we assume zero angular momentum ($L=0$).
:::

In [ ]:
from __future__ import annotations

import warnings

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import UnevaluatedExpression, implement_doit_method
from ampform.sympy.math import create_expression
from IPython.display import Math

warnings.filterwarnings("ignore")

## Square root definition

We define a special expression class for a square root where you can choose the sign for negative arguments. This can be used later for the phase space factor&nbsp;$\rho$.

In [ ]:
@implement_doit_method
class SignedSqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, z, **hints) -> SignedSqrt:
        return create_expression(cls, z, **hints)

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        return Rf"\sqrt[+]{{{z}}}"


@implement_doit_method
class PosArg(UnevaluatedExpression):
    is_commutative = True

    def __new__(cls, z, **hints) -> SignedSqrt:
        return create_expression(cls, z, **hints)

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg + 2 * sp.pi, sp.im(z) < 0),
            (arg, True),
        )

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        return Rf"\arg^+\left({z}\right)"


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

## Phase space factor definitions

### Standard phase space

In [ ]:
@implement_doit_method
class PhaseSpaceFactor(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m1, m2, **hints) -> PhspFactor:
        return create_expression(cls, s, m1, m2, *hints)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\rho^+\left({s}\right)"


s, m_a, m_b = sp.symbols("s m_a m_b")
rho_expr = PhaseSpaceFactor(s, m_a, m_b)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

### Chew-Mandelstamm function  

In [ ]:
@implement_doit_method
class BreakupMomentum(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m_a, m_b, **hints) -> PhspFactor:
        return create_expression(cls, s, m_a, m_b, **hints)

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (s * 4))

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"q^+\left({s}\right)"


breakup_momentum = BreakupMomentum(s, m_a, m_b)
Math(aslatex({breakup_momentum: breakup_momentum.doit(deep=False)}))

In [ ]:
@implement_doit_method
class ChewMandelstamm(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m1, m2, **hints) -> PhspFactor:
        return create_expression(cls, s, m1, m2, *hints)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q_break = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * (sp.pi) ** 2)
            * (
                (2 * q_break / sp.sqrt(s))
                * sp.log(
                    (m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q_break) / (2 * m1 * m2)
                )
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\rho^\text{{CM}}\left({s}\right)"


cm_expr = ChewMandelstamm(s, m_a, m_b)
Math(aslatex({cm_expr: cm_expr.doit(deep=False)}))

### Alternative definition of Chew-Mandelstamm

By J. L. Basdevant and Edmond L. Berger
(Phys. Rev. D 16, 657 – Published 1 August 1977)

In [ ]:
@implement_doit_method
class ChewMandelstammBB(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, s, m1, m2, **hints) -> ChewMandelstammBB:
        return create_expression(cls, s, m1, m2, *hints)

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        rho = PhaseSpaceFactor(s, m1, m2)
        return (
            1 / sp.pi
            + (m1**2 - m2**2) / (sp.pi * s) * sp.log(m2 / m1)
            - (m1**2 + m2**2) / (sp.pi * (m1**2 - m2**2)) * sp.log(m2 / m1)
            + rho
            / sp.pi
            * sp.log(
                (sp.sqrt((m1 + m2) ** 2 - s) - sp.sqrt((m1 - m2) ** 2 - s))
                / (sp.sqrt((m1 + m2) ** 2 - s) + sp.sqrt((m1 - m2) ** 2 - s))
            )
        )

    def _latex(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.args[0])
        return Rf"\rho^\text{{CM}}_\text{{BB}}\left({s}\right)"


cm_bb_expr = ChewMandelstammBB(s, m_a, m_b)
Math(aslatex({cm_bb_expr: cm_bb_expr.doit(deep=False)}))

### Visual comparison

In [ ]:
args = (s, m_a, m_b)
cm_func = sp.lambdify(args, cm_expr.doit())
cm_bb_func = sp.lambdify(args, cm_bb_expr.doit())
rho_func = sp.lambdify(args, rho_expr.doit())

s_values = np.linspace(0, 2, 400)
m1_val = 0.6
m2_val = 0.4
arg_vals = (m1_val, m2_val)

cm_values = 16 * np.pi * cm_func(s_values, *arg_vals)
cm_bb_values = cm_bb_func(s_values, *arg_vals)
rho_values = 1j * rho_func(s_values, *arg_vals)

fig, axes = plt.subplots(figsize=(15, 6), ncols=3, sharex=True, sharey=True)
ax1, ax2, ax3 = axes
ax1.plot(s_values, cm_values.real, label="Real part")
ax1.plot(s_values, cm_values.imag, label="Imaginary part")
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function value")
ax1.set_title("Chew Mandelstamm Function")
ax1.legend()

ax2.plot(s_values, rho_values.real, label="Real part")
ax2.plot(s_values, rho_values.imag, label="Imaginary part")
ax2.set_xlabel("Re(s)")
ax2.set_title("Phase Space Factor")
ax2.legend()

ax3.plot(s_values, cm_bb_values.real, label="Real part")
ax3.plot(s_values, cm_bb_values.imag, label="Imaginary part")
ax3.set_xlabel("Re(s)")
ax3.set_title("Chew Mandelstamm Function BB")
ax3.legend()

ax2.set_ylim(-1.2, +1.6)
plt.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

No Blatt-Weisskopf-Formfactors because $L=0$:

In [ ]:
gamma_R, m_R, w_R = sp.symbols("gamma_R m_R Gamma_R")
k_expr = (gamma_R**2 * m_R * w_R) / (s - m_R**2)

In [ ]:
rho_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: PhaseSpaceFactor(s, m_a, m_b),
}
Math(aslatex(rho_expressions))

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: -sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a, m_b),
}
Math(aslatex(cm_expressions))

In [ ]:
cm_conj_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: -sp.I * 16 * sp.pi * ChewMandelstamm(s, m_a, m_b).conjugate(),
}
Math(aslatex(cm_conj_expressions))

In [ ]:
T_rho_expr = T_explicit[0, 0].xreplace(rho_expressions)
T_cm_expr = T_explicit[0, 0].xreplace(cm_expressions)
T_cm_conj_expr = T_explicit[0, 0].xreplace(cm_conj_expressions)

In [ ]:
T_cm_expr

## Riemann sheet visualization

To get to the physical/unpysical Riemann sheet one uses XXXX which is derived from Schwarz reflection priciple

In [ ]:
args = (s, m_a, m_b, m_R, w_R, gamma_R)
T_rho_func = sp.lambdify(args, T_rho_expr.doit())
T_cm_func = sp.lambdify(args, T_cm_expr.doit())
T_cm_conj_func = sp.lambdify(args, T_cm_conj_expr.doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 2.2, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Yn = np.meshgrid(x, +y)
_, Yp = np.meshgrid(x, -y)
Zn = X + Yn * 1j
Zp = X + Yp * 1j

m1_val = 0.25
m2_val = 0.45
mR_val = 1.1
wR_val = 0.3
gamma_val = 1
arg_vals = (m1_val, m2_val, mR_val, wR_val, gamma_val)

Tp = T_rho_func(Zp**2, *arg_vals)
Tn = T_rho_func(Zn**2, *arg_vals)

Tp_cm = T_cm_func(Zp**2, *arg_vals)
Tn_cm = T_cm_func(Zn**2, *arg_vals)

Tp_cm_conj = T_cm_conj_func(Zp**2, *arg_vals)
Tn_cm_conj = T_cm_conj_func(Zn**2, *arg_vals)

In [ ]:
fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharey=True)
ax1, ax2 = axes

ax1.plot(x, Tp_cm[0].imag)
ax1.plot(x, Tn_cm_conj[0].imag)
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function Value")
ax1.set_title("Chew Mandelstamm Function")

ax2.plot(x, Tp[0].imag)
ax2.plot(x, Tn[0].imag)
ax2.set_xlabel("Re(s)")
ax2.set_title("Phsp Function")

plt.tight_layout()
plt.show()

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        name=sheet_name,
        opacity=opacity,
        showscale=False,
    )


opacity = 0.8
vmax = 0.8
project = np.imag
projection_text = "Imaginary"
sheet_colors = {
    "Unphysical": "red",
    "Physical": "blue",
}

Sn = go.Surface(x=X, y=Yn, z=project(Tn), **sty("Unphysical"))
Sp = go.Surface(x=X, y=Yp, z=project(Tp), **sty("Physical"))

Sn_cm = go.Surface(x=X, y=Yn, z=project(Tn_cm), **sty("Unphysical"))
Sp_cm = go.Surface(x=X, y=Yp, z=project(Tp_cm), **sty("Physical"))

Sn_cm_conj = go.Surface(x=X, y=Yn, z=project(Tn_cm_conj), **sty("Unphysical"))
Sp_cm_conj = go.Surface(x=X, y=Yp, z=project(Tp_cm_conj), **sty("Physical"))

zeros = np.zeros(x.shape)
line_rho = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(Tp[0]),
    line={"color": "black", "width": 4},
    mode="lines",
    name="Lineshape",
)
line_cm = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(Tn_cm_conj[0]),
    line={"color": "black", "width": 4},
    mode="lines",
    name="Lineshape",
)

threshold_point = go.Scatter3d(
    x=[(m1_val + m2_val) ** 2],
    y=[0],
    z=[0],
    mode="markers",
    marker={"color": "black", "size": 5},
    name="Threshold",
)

fig_rho = go.Figure(data=[Sn, Sp, line_rho, threshold_point])
fig_rho.update_layout(
    height=550,
    width=600,
    showlegend=False,
    title_text=(
        f"{projection_text} part of T function with standard phase space factor"
    ),
)
fig_rho.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig_cm = go.Figure(data=[Sn_cm_conj, Sp_cm, line_cm, threshold_point])
fig_cm.update_layout(
    height=550,
    width=600,
    showlegend=False,
    title_text=f"{projection_text} part of T function with Chew-Mandelstam",
)
fig_cm.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig_rho.show()
fig_cm.show()